1.	[Data Engineering] To build the dashboard you would first need to source the economic data. Releases of the data can be found on the Federal Reserve Economic Data (FRED), the Office for National Statistics (ONS) and the National Bureau of Statistics of China databases for the US, UK and China respectively. Retrieve data from these sources for the 3 countries for a period of your choice to be displayed in the dashboard. Please provide the series IDs of / links to the datasets you obtained, so that the portfolio manager can trace the data back to their original sources.

# LINKS FOR DATA

https://fred.stlouisfed.org/series/GDPC1 - us real gdp
https://fred.stlouisfed.org/series/MRTSSM44X72USS - us retail sales
https://fred.stlouisfed.org/series/INDPRO - us industrial production


https://fred.stlouisfed.org/series/CLVMNACSCAB1GQUK - uk real gdp
https://fred.stlouisfed.org/series/GBRSLRTTO02IXOBSAM - uk retail sales
https://fred.stlouisfed.org/series/IPIUKM - uk industrial production

https://fred.stlouisfed.org/series/CHNGDPNQDSMEI - china NOMINAL gdp
https://fred.stlouisfed.org/series/CHNSLRTTO02MLSAM - china retail sales
https://fred.stlouisfed.org/series/PRCNTO01CNQ661S china industrial production

I decided to use the same source for all the data in order to ensure standardisation of format and calculation practices and standards. Also, it was quite difficult to find the relevant data from the National Bureau of Statistics of China. However, one thing to note was that for industrial production, the data for US and UK is an index but that is not the case for the data about China. Also, there are varying units for the different data but since the heatmap is to show the percentage changes QoQ and YoY, this should not be a big issue

# Loading the data

In [8]:
import pandas as pd
usGDP = pd.read_excel("usgdp.xls") #(bil)
usGDP
usRS = pd.read_excel("USRS.xls") #(mil)
usIDP = pd.read_excel("USIDP.xls") #(index)

ukGDP = pd.read_excel("UKGDP.xls") #(mil)
ukRS = pd.read_excel("UKRS.xls") #(index)
ukIDP = pd.read_excel("UKIDP.xls") #(index)

chiGDP = pd.read_excel("CHIGDP.xls")
chiRS = pd.read_excel("CHIRS.xls")
chiIDP = pd.read_excel("CHIIDP.xls") #(index)

# Preprocessing the data

In [9]:
#initialise first and last dates, to take the latest for the start dates and earliest for the end dates so that we can use
#the most data for dates that have all the data
start_dates = []
end_dates = []

#here i went through each metric individually to examine the structure of the data
usGDP.head(15) # first datapoint on row 10, 1947-01-01, quarterly
usGDP.tail(5) # last datapoint 2023-04-01
start_dates.append("1947-01-01")
end_dates.append("2023-04-01")
usGDP = usGDP[10:]
usGDP = usGDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "usGDP"})
# usGDP["usGDP"] = usGDP.apply(lambda row: row["usGDP"] * 1000000000) #in billions
usGDP

usRS.head(15) # first datapoint on row 10, 1992-01-01, monthly
usRS.tail(5) # last datapoint 2023-08-01
start_dates.append("1992-01-01")
end_dates.append("2023-08-01")
usRS = usRS[10:]
usRS = usRS.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "usRS"})
usRS

usIDP.head(15) # first datapoint on row 10, 1919-01-01, monthly
usIDP.tail(5) # last datapoint 2023-09-01
start_dates.append("1919-01-01")
end_dates.append("2023-09-01")
usIDP = usIDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "usIDP"})
usIDP = usIDP[10:]
usIDP

ukGDP.head(15) #first datapoint row 10, 1975-01-01 quarterly
ukGDP.tail(5) # 2020-07-01
start_dates.append("1975-01-01")
end_dates.append("2020-07-01")
ukGDP = ukGDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "ukGDP"})
ukGDP = ukGDP[10:]
ukGDP

ukRS.head(15) #first datapoint row 10, 1992-01-01 monthly
ukRS.tail(5) # 2023-08-01
start_dates.append("1992-01-01")
end_dates.append("2023-08-01")
ukRS = ukRS.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "ukRS"})
ukRS = ukRS[10:]
ukRS

# ukIDP.head(15) #first datapoint row 10, 1920-01-01 monthly
# ukIDP.tail(5) # 2017-01-01
start_dates.append("1920-01-01")
end_dates.append("2017-01-01")
ukIDP = ukIDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "ukIDP"})
ukIDP = ukIDP[10:]

# chGDP.head(15) #first datapoint row 10, 1992-01-01 quarterly
# chGDP.tail(5) # 2023-04-01
start_dates.append("1992-01-01")
end_dates.append("2023-04-01")
chiGDP = chiGDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "chiGDP"})
chiGDP = chiGDP[10:]
chiGDP 


chiRS.head(15) #first datapoint row 10, 1993-01-01 monthly
chiRS.tail(5) # 2018-12-01
start_dates.append("1993-01-01")
end_dates.append("2018-12-01")
chiRS = chiRS.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "chiRS"})
chiRS = chiRS[10:]
chiRS

chiIDP.head(15) #first datapoint row 10, 1996-01-01 monthly
chiIDP.tail(5) #2023-04-01 
start_dates.append("1996-01-01")
end_dates.append("2023-04-01 ")
chiIDP = chiIDP.rename(columns = {"FRED Graph Observations" : "Date", "Unnamed: 1" : "chiIDP"})
chiIDP = chiIDP[10:]
chiIDP

,Date,chiIDP
10,1996-01-01 00:00:00,10.54339
11,1996-04-01 00:00:00,10.387057
12,1996-07-01 00:00:00,9.898356
13,1996-10-01 00:00:00,10.412359
14,1997-01-01 00:00:00,10.590745
...,...,...
115,2022-04-01 00:00:00,90.445906
116,2022-07-01 00:00:00,95.319819
117,2022-10-01 00:00:00,97.186978
118,2023-01-01 00:00:00,106.634827


In [10]:
start_date = max(start_dates)
end_date = min(end_dates)
print(start_date, end_date)

1996-01-01 2017-01-01


2.	[Data Visualisation] The portfolio manager would like you to build an interactive heatmap with accompanying widgets so that the output of the heatmap is customisable depending on the country, economic indicator, data transformation, end period, lookback period and number of quantiles chosen (see Appendix for details).

# Converting monthly data to quarterly data

In [11]:
target_dates = [(1, 1), (4, 1), (7, 1), (10, 1)]
usGDP.iloc[0]["Date"]
for df in [usRS,usIDP,ukRS,ukIDP,chiRS,chiIDP]:
    df['Date'] = pd.to_datetime(df['Date'])  # Convert the 'Date' column to datetime if it's not already
    
    # Create a new column 'Month_Day' containing the month and day
    df['Month_Day'] = df['Date'].dt.strftime('%m-%d')
    
    # Use the .isin() method to filter rows with matching dates in place
    df.drop(df[~df['Month_Day'].isin(['{:02d}-{:02d}'.format(month, day) for month, day in target_dates])].index, inplace=True)
    
    # Drop the 'Month_Day' column in place
    df.drop(columns=['Month_Day'], inplace=True)


# filtering all the data to be within the specified time frame of 1996-01-01 to 2017-01-01

In [12]:
start_date = pd.to_datetime('1996-01-01')
end_date = pd.to_datetime('2017-01-01')
all_dfs = [usRS,usIDP,ukRS,ukIDP,chiRS,chiIDP, usGDP,ukGDP,chiGDP]

for df in all_dfs:
    df['Date'] = pd.to_datetime(df['Date'])  # Convert the 'Date' column to datetime if it's not already
    
    # Filter the DataFrame in place to include only data within the specified time frame
    df.drop(df[(df['Date'] < start_date) | (df['Date'] > end_date)].index, inplace=True)

# merging the dataframes

In [13]:
combined_df = all_dfs[0]  # Start with the first DataFrame
# Merge the remaining DataFrames one by one
for df in all_dfs[1:]:
    combined_df = pd.merge(combined_df, df, on='Date', how='outer')

In [14]:
from ipywidgets import interact, widgets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def update_heatmap(Countries, Indicator, Transformation, Lookback_periods, End_period, Number_of_quantiles):
    if End_period is not None:
        end_date = pd.to_datetime(End_period)
        # Check if the selected end_date is after '2017-01-01'
        if end_date > pd.to_datetime('2017-01-01'):
            print("End date cannot be after '2017-01-01'.")
            return
    else:
        end_date = pd.to_datetime('2017-01-01')  # Provide a default end date if End_period is not selected

    if not Countries:
        print("Please select Country")
        return

    # Calculate the start date based on the number of quarters
    if Transformation == "Quarter-on-Quarter":
        start_date = end_date - pd.DateOffset(months=Lookback_periods * 3)
    else:
        start_date = end_date - pd.DateOffset(months=Lookback_periods * 12)
    # Filter data based on lookback periods and end period
    filtered_df = combined_df[(combined_df["Date"] >= start_date)  & (combined_df["Date"] <= end_date)]

    # Calculate heatmap values for selected countries
    heatmap_data = []
    for country in Countries:
        # Calculate the values based on Indicator, Transformation, and selected country
        # You may need to replace this with your specific calculation logic
        if Indicator == "Real_GDP":
            data = filtered_df[["Date", f"{country.lower()}GDP"]]
        elif Indicator == "Retail_Sales":
            data = filtered_df[["Date",f"{country.lower()}RS"]]
        elif Indicator == "Industrial_Production":
            data = filtered_df[["Date",f"{country.lower()}IDP"]]

        if Transformation == "Year-on-Year":
            # Calculate year-on-year values by dropping all other rows that are not yoy data
            # Use the .dt accessor to extract the month and day as integers
            target_month = end_date.month
            
            #if target_month is not a in quarter, find the nearest quarter
            while target_month not in [10,7,4,1]:
                target_month -= 1

            # Filter the DataFrame
            data = data[data['Date'].dt.month == target_month]
            values = data.iloc[:, 1].pct_change(periods=1) * 100
            
        elif Transformation == "Quarter-on-Quarter":
            # Calculate quarter-on-quarter values
            values = data.iloc[:, 1].pct_change(periods=1) * 100

        heatmap_data.append(values)

    # Create the heatmap with the correct number of quantiles and colors
    n_colors = Number_of_quantiles  # Match the number of colors to the number of quantiles
    cmap = sns.color_palette("coolwarm", n_colors)  # Use the color palette for coolwarm with the desired number of colors
    
    # Convert dates to strings for labeling
    x_labels = [str(date.date()) for date in data["Date"]]
    y_labels = list(Countries)

    plt.figure(figsize=(16, 12))  # Increase the figure size for better spacing
    ax = sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap=cmap, linewidths=0.5, cbar_kws={'label': f'{n_colors} Quantiles', 'ticks': range(n_colors)})
    
    ax.set_xticklabels(x_labels, rotation=45, fontsize=12)  # Adjust font size and rotation
    ax.set_yticklabels(y_labels, rotation=0, fontsize=12)  # Adjust font size

    country_names = ', '.join(Countries)
    plt.title(f'{country_names} - {Indicator} - {Transformation} - {Lookback_periods} {Transformation.split("-")[-1]} ({start_date.date()} to {end_date.date()})', fontsize=16)  # Adjust title font size
    plt.show()

# Create interactive widgets with label width adjustment
Country_widget = widgets.SelectMultiple(options=['US', 'UK', 'CHI'], description='Country:', layout=widgets.Layout(width='300px'))
Indicator_widget = widgets.Dropdown(options=['Real_GDP', 'Retail_Sales', 'Industrial_Production'], description='Indicator:', layout=widgets.Layout(width='300px'))
Transformation_widget = widgets.Dropdown(options=['Year-on-Year', 'Quarter-on-Quarter'], description='Transformation:', layout=widgets.Layout(width='300px'))
Lookback_periods_widget = widgets.BoundedIntText(min=1, description='Lookback periods:', layout=widgets.Layout(width='300px'))
End_period_widget = widgets.DatePicker(description='End period:', layout=widgets.Layout(width='300px'))
Number_of_quantiles_widget = widgets.IntSlider(min=3, max=5, step=2, description='Quantiles:', layout=widgets.Layout(width='300px'))

# Create the interactive heatmap
interact(update_heatmap, Countries=Country_widget, Indicator=Indicator_widget, Transformation=Transformation_widget,
         Lookback_periods=Lookback_periods_widget, End_period=End_period_widget, Number_of_quantiles=Number_of_quantiles_widget)


interactive(children=(SelectMultiple(description='Country:', layout=Layout(width='300px'), options=('US', 'UK'…

<function __main__.update_heatmap(Countries, Indicator, Transformation, Lookback_periods, End_period, Number_of_quantiles)>

# Note for the interactive heatmap: 
# multiselect for countries can be done by ctrl click
# default end period is 2017-01-01

3.	[Data Analysis / Forecasting] Using the retrieved data and built dashboard in Questions 1 & 2, provide a short write-up analysing the recent trends in the economic indicators that you observe, and their investment implications (e.g., Are any countries facing a technical recession? i.e., 2 consecutive quarters of negative Real GDP growth). You may optionally build a model to forecast the year-on-year % changes of Real GDP for 2024 to add to your analysis (you may add any variables and use any model you deem appropriate for the forecast).

# GDP DATA

As compared to US and UK, the percentage changes in the real GDP growth are much more drastic. Although this can be attributed to the GDP data of China being nominal as compared to real GDP of US and UK accounted for inflation, this can signal more volatility in investments in china given this trend in its macroeconomic condition. However, this may also just be due to the type of GDP data that China has since it is GDP by expenditure due to the limitation of the data that can be found.

As expected, US and UK experienced technical recessions in 2008 due to the global financial crisis, but it was quite interesting to see how the nominal GDP for china fluctuated during this period, assuming that they measure similar metrics

# Retail sales data

It can be unfair to make comparisons since the data for retail sales is not exactly the same with different variants of retail sales and trade, as well as the UK data being an index instead of the absolute value.

However if we make the assumption that they can be compared this way, the YoY and QoQ growth in retail sales in China greatly outperforms that in UK and US. This may be partly due to UK and US being more mature economies, as well as the great increase in purchasing power of the Chinese in the recent years with the opening and expansion of their economy.

In terms of investment implications, there may be more opportunities in this particular sector for the chinese economy due to its rapid growth

# Industrial production data

Once again it is unfair to make comparisons since the data for UK and US are indexes but not for that in China. Overall the FRED website is not very robust and comprehensive in its chinese data but there was a lack of historical data on the National Bureau of Statistics of China databases as well.

However if we were to make the assumption again, there is higher growth in the chinese economy for the industrial production, possibly due to the size of its population and workforce which contributes to its high industrial production.

In terms of investment implications, there may be more opportunities in this particular sector for the chinese economy due to its rapid growth.


# Concluding statement

It is evident that it is imperative to obtain the most similar types of data in the same format eg. indexes or value in order to make meaningful and fair comparisons from which investment decisions can be made.

Also, these metrics are not solely economic but they are affected by political, geographical, and social factors as well. By analysing their patterns in combination with research on these other factors that can affect the overall macroeconomic environment, more informed investment decisions can be made with the help of data analytics and science.